In [1]:
%%markdown 
# Data Engineering 

# Data Engineering 


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os 
import seaborn as sns 
import sklearn as skl
from imblearn.over_sampling import SMOTE
from scipy.stats import norm 
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler 
import warnings  
warnings.filterwarnings('ignore') 
%matplotlib inline 

In [3]:
data = pd.read_csv('../../data/watson_marketing.csv')
data.Response = data.Response.apply(lambda x: 0 if x=='No' else 1)
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,0,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,0,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,0,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,0,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,0,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
data.groupby(['Response']).count()['Customer']

Response
0    7826
1    1308
Name: Customer, dtype: int64

In [5]:
%%markdown 
### The number of people who responded to the campaign is considerably lower than the number who did not. This could lead to an undersampling issue which needs to be taken care of. 


### To deal with the undersampling I have decided to use SMOTE. This is will lead to synthetic data being created for the undersamplingdersampled class. 

### The number of people who responded to the campaign is considerably lower than the number who did not. This could lead to an undersampling issue which needs to be taken care of. 


### To deal with the undersampling I have decided to use SMOTE. This is will lead to synthetic data being created for the undersamplingdersampled class. 


In [6]:
%%markdown

## Standardizing continuous Data. 


## Standardizing continuous Data. 


In [7]:
X_data_all = data.drop(['Response'],axis=1)
Y_data = data['Response']

In [8]:
X_data_numeric = X_data_all.select_dtypes(include=['int64','float'])
X_data_numeric.nunique()

Customer Lifetime Value          8041
Income                           5694
Monthly Premium Auto              202
Months Since Last Claim            36
Months Since Policy Inception     100
Number of Open Complaints           6
Number of Policies                  9
Total Claim Amount               5106
dtype: int64

In [9]:
columns = X_data_numeric.columns
ss = StandardScaler()
X_data_numeric_normalized = ss.fit_transform(X_data_numeric)
X_data_numeric_normalized = pd.DataFrame(X_data_numeric_normalized,columns=columns)

In [10]:
%%markdown

## Encoding Categorical Data. 


## Encoding Categorical Data. 


In [11]:
categorical_df = X_data_all.select_dtypes(include='object')
cat_df = categorical_df.drop(['Customer','Effective To Date'], axis = 1)
cat_df.nunique()

State               5
Coverage            3
Education           5
EmploymentStatus    5
Gender              2
Location Code       3
Marital Status      3
Policy Type         3
Policy              9
Renew Offer Type    4
Sales Channel       4
Vehicle Class       6
Vehicle Size        3
dtype: int64

In [12]:
cat_columns = cat_df.columns
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
for col in cat_df[cat_columns]:
    cat_df[col] = lb.fit_transform(cat_df[col])
    cat_df[col] = cat_df[col].astype('category')

cat_df = pd.get_dummies(cat_df,drop_first=True)
cat_df.head()

,State_1,State_2,State_3,State_4,Coverage_1,Coverage_2,Education_1,Education_2,Education_3,Education_4,...,Sales Channel_1,Sales Channel_2,Sales Channel_3,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,Vehicle Size_1,Vehicle Size_2
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
%%markdown 
## Merging engineered continuious and categorical features. 

## Merging engineered continuious and categorical features. 


In [14]:
data_engineered_merged = pd.concat([X_data_numeric_normalized.reset_index(drop=True),cat_df.reset_index(drop=True)], axis=1)
data_engineered_merged.head()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,State_1,State_2,...,Sales Channel_1,Sales Channel_2,Sales Channel_3,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,Vehicle Size_1,Vehicle Size_2
0,-0.762878,0.612827,-0.703925,1.678099,-1.543287,-0.42225,-0.822648,-0.169640,0,0,...,0,0,0,0,0,0,0,1,1,0
1,-0.149245,-1.239617,0.022691,-0.208186,-0.217334,-0.42225,2.106160,2.400737,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.710636,0.365710,0.429596,0.288205,-0.360680,-0.42225,-0.404247,0.455734,0,1,...,0,0,0,0,0,0,0,1,1,0
3,-0.052263,-1.239617,0.371467,0.288205,0.606907,-0.42225,1.687759,0.329769,1,0,...,0,1,0,0,0,1,0,0,1,0
4,-0.755575,0.203390,-0.587666,-0.307465,-0.145661,-0.42225,-0.822648,-1.018843,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
total_train = sm.Logit(Y_data, data_engineered_merged)
total_train.fit().summary()

         Current function value: 0.320411
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Response   No. Observations:                 9134
Model:                          Logit   Df Residuals:                     9086
Method:                           MLE   Df Model:                           47
Date:                Mon, 14 Sep 2020   Pseudo R-squ.:                  0.2199
Time:                        14:30:08   Log-Likelihood:                -2926.6
converged:                      False   LL-Null:                       -3751.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Customer Lifetime Value          -0.0313      0.037     -0.843      0.399      -0.104       0.041
Income                            0.1781      0.058      3.045      0.002       0.063       0.293
Monthly Premium Auto              0.4789      0.164      2.926      0.003       0.158       0.800
Months Since Last Claim          -0.0480      0.034     -1.407      0.159      -0.115       0.019
Months Since Policy Inception    -0.0098      0.034     -0.294      0.769      -0.076       0.056
Number of Open Complaints        -0.0341      0.035     -0.987      0.324      -0.102       0.034
Number of Policies               -0.0504      0.034     -1.486      0.137      -0.117       0.016
Total Claim Amount               -0.2860      0.076     -3.772      0.000      -0.435      -0.137
State_1                          -0.0892      0.093     -0.956      0.339      -0.272       0.094
State_2                          -0.0863      0.132     -0.656      0.512      -0.344       0.172
State_3                          -0.0647      0.097     -0.668      0.504      -0.255       0.125
State_4                          -0.1718      0.136     -1.259      0.208      -0.439       0.096
Coverage_1                       -0.2978      0.118     -2.528      0.011      -0.529      -0.067
Coverage_2                       -0.4409      0.243     -1.811      0.070      -0.918       0.036
Education_1                       0.1106      0.086      1.280      0.201      -0.059       0.280
Education_2                       0.4485      0.170      2.630      0.009       0.114       0.783
Education_3                      -0.1080      0.089     -1.214      0.225      -0.282       0.066
Education_4                       0.3278      0.128      2.565      0.010       0.077       0.578
EmploymentStatus_1               -0.5960      0.142     -4.197      0.000      -0.874      -0.318
EmploymentStatus_2               -0.2054      0.179     -1.149      0.251      -0.556       0.145
EmploymentStatus_3                2.1927      0.200     10.939      0.000       1.800       2.586
EmploymentStatus_4               -0.9595      0.151     -6.342      0.000      -1.256      -0.663
Gender_1                         -0.0275      0.068     -0.407      0.684      -0.160       0.105
Location Code_1                   1.1506      0.133      8.658      0.000       0.890       1.411
Location Code_2                  -0.0141      0.134     -0.105      0.916      -0.276       0.248
Marital Status_1                 -0.6642      0.086     -7.699      0.000      -0.833      -0.495
Marital Status_2                 -0.6410      0.104     -6.164      0.000      -0.845      -0.437
Policy Type_1                    -0.3698   4.04e+06  -9.15e-08      1.000   -7.92e+06    7.92e+06
Policy Type_2                    -0.2144   9.93e+06  -2.16e-08      1.000   -1.95e+07    1.95e+07
Policy_1                         -0.2813      0.187     -1.503      0.133      -0.648       0.085
Policy_

In [16]:
%%markdown

## Filtering continuous data for significant features 

Significant features are the ones with the p value from the regression analysis which are less than 0.05. Our significant features are:
    - Income 
    - Monthly Premium Auto 
    - Total Claim Amount 
    - Coverage
    - Education 
    - Employment Status 
    - Location Code 
    - Marital Status
    - Policy    
    - Renew Offer Type 
    - Sales Channel 
    - Vehicle Class 
    - Vehicle Size 


## Filtering continuous data for significant features 

Significant features are the ones with the p value from the regression analysis which are less than 0.05. Our significant features are:
    - Income 
    - Monthly Premium Auto 
    - Total Claim Amount 
    - Coverage
    - Education 
    - Employment Status 
    - Location Code 
    - Marital Status
    - Policy    
    - Renew Offer Type 
    - Sales Channel 
    - Vehicle Class 
    - Vehicle Size 


In [17]:
X_data_numeric_normalized_significant = X_data_numeric_normalized[['Income','Monthly Premium Auto','Total Claim Amount']]
X_data_numeric_normalized_significant.head()

,Income,Monthly Premium Auto,Total Claim Amount
0,0.612827,-0.703925,-0.169640
1,-1.239617,0.022691,2.400737
2,0.365710,0.429596,0.455734
3,-1.239617,0.371467,0.329769
4,0.203390,-0.587666,-1.018843


In [18]:
%%markdown

## Filtering categorical data for significant features 


## Filtering categorical data for significant features 


In [19]:
categorical_df = X_data_all.select_dtypes(include='object')
cat_df = categorical_df.drop(['Customer','Effective To Date'], axis = 1)
cat_df_significant = cat_df[['Coverage','Education','EmploymentStatus','Location Code','Marital Status','Policy','Renew Offer Type','Vehicle Class','Vehicle Size']]
cat_columns_significant = cat_df_significant.columns
lb = LabelEncoder()
for col in cat_df_significant[cat_columns_significant]:
    cat_df_significant[col] = lb.fit_transform(cat_df_significant[col])
    cat_df_significant[col] = cat_df_significant[col].astype('category')

cat_df_significant = pd.get_dummies(cat_df_significant,drop_first=True)
cat_df_significant.head()

,Coverage_1,Coverage_2,Education_1,Education_2,Education_3,Education_4,EmploymentStatus_1,EmploymentStatus_2,EmploymentStatus_3,EmploymentStatus_4,...,Renew Offer Type_1,Renew Offer Type_2,Renew Offer Type_3,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,Vehicle Size_1,Vehicle Size_2
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
data_engineered_merged_significant = pd.concat([X_data_numeric_normalized_significant.reset_index(drop=True),cat_df_significant.reset_index(drop=True)], axis=1)

,Income,Monthly Premium Auto,Total Claim Amount,Coverage_1,Coverage_2,Education_1,Education_2,Education_3,Education_4,EmploymentStatus_1,...,Renew Offer Type_2,Renew Offer Type_3,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,Vehicle Size_1,Vehicle Size_2,Response
0,0.612827,-0.703925,-0.169640,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
1,-1.239617,0.022691,2.400737,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,0.365710,0.429596,0.455734,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
3,-1.239617,0.371467,0.329769,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0.203390,-0.587666,-1.018843,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [21]:
data_engineered_merged_significant.to_csv('created_data/feature_engineered_data.csv',index=False)